# Debugging A Simple Image Processing Program Using The Compute Sanitizer
## 06 Initcheck

In this section, we will introduce the initcheck tool. Initcheck is designed to
detect and report the usage of device memory that has not been initialized.
Reading uninitialized values means that the value read will be unpredictable,
most likely leading to unexpected results within the application.

Now that our application seems to have correct results in our smaller square
test images, let's try it on a bigger scale. Uncomment the part at the bottom of
the file, between lines 255 and 275. This part adds the application of a Gaussian
blur to the [coffee.png](code/coffee.png) beans image. The expected result is the following:

![Coffe Beans blur](images/coffee-refscaled.png)

Rebuild and run the `convolution` application. We can observe that instead of
the expected result for [coffee-out.png](code/coffee-out.png) we have an almost black picture.

![Dark coffee](images/dark-coffeescaled.png)

First, try to reuse the tools that we already ran to see if any new errors
appeared.

Once that's done, we can observe that no new errors are reported by memcheck or
racecheck. Therefore, we will now try to see if initcheck can help us debug the
application:

```sh
    compute-sanitizer --tool=initcheck ./convolution
```

Here, we can observe that the tool reports several errors. Given the number of errors reported, the performance of the program is heavily impacted and you may interrupt its execution by pressing Ctrl+C and start debugging.
Please take a moment to examine these reports and try to find the error.

```
========= COMPUTE-SANITIZER
========= Uninitialized __global__ memory read of size 1 bytes
=========     at 0xc40 in /home/nvidia/sanitizer-demo/convolution.cu:48:convolution(pixel const*, int, int, float const*, int, pixel*)
=========     by thread (3,3,0) in block (0,0,0)
=========     Address 0x7fa8e5002048
=========     Saved host backtrace up to driver entry point at kernel launch time
=========     Host Frame: [0x27e53a]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x108db]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x5c300]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb1e8]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__device_stub__Z11convolutionPK5pixeliiPKfiPS_(pixel const *, int, int, float const *, int, pixel*) [0xb025]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:convolution(pixel const *, int, int, float const *, int, pixel*) [0xb092]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xa298]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xac48]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
[...]
```

The error is located line 269: instead of copying the full convolution matrix,
we only copy the 25 first bytes, leading to the remaining bytes being
uninitialized but still read by the kernel. Possible fixes are to multiply the
`25` by `sizeof(float)` or to directly use `sizeof(gaussianBlurMatrix)` instead.

After fixing the mistake, recompile and launch the application again. We now
have the expected result, a slight blur on our [coffee-out.png](code/coffee-out.png) beans picture. The effect is light so it may be hard to notice.
This shows that even if the device code is correct, error in handling the CUDA
API can still lead failures that may be non-trivial to detect with a regular
debugger.

The application now seems to be working correctly. However, there may be some
things that can still be improved that the compute-sanitizer could help us with.
    
[Continue to step 07.](07_memory_leaks.ipynb)
